### Create a pdf of $\Delta p_{T} / p_{T}^{2}$ histograms for lep1 and lep2 in various $\eta$ bins and $p_{T}$ bins.

A couple of notes:
- Check the masks: may only select the `GENmass2l==0`
- Still need to implement `60 < massZ < 120`

In [ ]:
import sys
import os
sys.path.append('/Users/Jake/anaconda3/lib/python3.7/site-packages')

import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# import ROOT
# import root_numpy
sys.path.append('/Users/Jake/')
sys.path.append('/Users/Jake/HiggsMassMeasurement/')
sys.path.append('/Users/Jake/HiggsMassMeasurement/d0_Studies/')

# Neat tricks.
from matplotlib.backends.backend_pdf import PdfPages
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Local imports. 
from PyUtils.fileUtils import makeDirs, copyFile
# from PyUtils.tdrStyle import setTDRStyle
from d0_Utils.d0_fns import (combine_cut_list, calc_num_bins, make_str_title_friendly, 
                             calc_x_err_bins, calc_ymin_for_legend, print_header_message,
                             add_underoverflow_entries)
from d0_Utils.d0_dicts import (charge_dict, color_dict)
# from d0_Utils.d0_Analyzer_with_fancy_functions import HistInfo

# pd.options.display.max_columns = 23
pd.options.display.max_columns = None

plt.style.use('cmsstyle_plot')

In [ ]:
infile_path = '/Users/Jake/Desktop/Research/Higgs_Mass_Measurement/NTuples/MC/MC_2016.feather'
df = pd.read_feather(infile_path)

In [ ]:
pt_range_ls = [
    [5,20],
    [20,30],
    [30,40],
    [40,50],
    [50,60],
    [60,100]
]

eta_range_ls =[
    [0.0, 0.8],
    [0.8, 1.8],
    [1.8, 2.4],
]

n_evts = -1  # Set to -1 to run over all events. 
xvals = [-0.04, 0.04]
bwidth = 0.001

if n_evts == -1:
    n_evts = len(df)

df = df[:n_evts]
year = "2016"
extra_filename = ""
outpath = '/Users/Jake/Desktop/'
datatype = "MC"
save_plots = True

#--- Automatons ---#
# Dynamic numbering of subplots.
n_plots = len(pt_range_ls)
rows = int(n_plots / 2)
cols = int(n_plots / rows)

xmin = xvals[0]
xmax = xvals[1]
    
mask_badGENmass2l = df['GENmass2l'] == 0
mask_goodGENmass2l = df['GENmass2l'] > 0

#--- Plot the thang ---#
filename = f'dpT_dist_pTbinning__{datatype}_{year}__0p0_eta_2p4'
filename += "_" + extra_filename
filename = make_str_title_friendly(filename) #+ "__" + extra_filename
fullpath = os.path.join(outpath, filename)
with PdfPages(fullpath) as pdf:

    for eta_ls in eta_range_ls:

        eta_min = eta_ls[0]
        eta_max = eta_ls[1]

        # Make a dpT plot for lep1 and lep2 in the different pT regions.
        fig = plt.figure(figsize=(12,12))

        for ct,pT_ls in enumerate(pt_range_ls, start=1):

            pT_min = pT_ls[0]
            pT_max = pT_ls[1]

            # Masks.
            mask_eta1 = (eta_min < df['eta1']) & (df['eta1'] < eta_max)
            mask_eta2 = (eta_min < df['eta2']) & (df['eta2'] < eta_max)

            mask_pT1 = (pT_min < df['pT1']) & (df['pT1'] < pT_max)
            mask_pT2 = (pT_min < df['pT2']) & (df['pT2'] < pT_max)

            # Apply masks to make new DF.
            df_bad_GENmass2l_lep1 = df[mask_badGENmass2l & mask_pT1 & mask_eta1]
            df_bad_GENmass2l_lep2 = df[mask_badGENmass2l & mask_pT2 & mask_eta2]

            data1 = (df_bad_GENmass2l_lep1['pT1'] - df_bad_GENmass2l_lep1['genLep_pt1']) / df_bad_GENmass2l_lep1['genLep_pt1']**2
            data2 = (df_bad_GENmass2l_lep2['pT2'] - df_bad_GENmass2l_lep2['genLep_pt2']) / df_bad_GENmass2l_lep2['genLep_pt2']**2

            # Account for under-overflow data.
            overflow_data1 = add_underoverflow_entries(data1, xmin, xmax)
            overflow_data2 = add_underoverflow_entries(data2, xmin, xmax)

            # plt.subplot may be the way to go! It allows minor ticks to show up reliably.
            # However, you just give it an index to specify the axis you want. This gives you less control.
            this_ax = plt.subplot(rows, cols, ct)
            this_ax.set_xlim([xmin,xmax])
            this_ax.set_xlabel(r"$(p_{T}^{RECO} - p_{T}^{GEN}) / (p_{T}^{GEN})^2$ [GeV]$^{-1}$")#, fontsize=textsize_axislabels)
            this_ax.set_ylabel(f"Events / [{bwidth:.1E} GeV" + r"$^{-1}$]")#, fontsize=textsize_axislabels)
            this_ax.set_title(r"GENmass2$\ell$==0,  "
                              f"{eta_min}" + r"$< \eta <$" + f"{eta_max},  "
                              f"{pT_min} GeV" + r"$<p_{T}<$" + f"{pT_max} GeV")#, fontsize=textsize_title)

            n1, bins1, _ = this_ax.hist(overflow_data1, bins=np.linspace(xmin,xmax,int((xmax-xmin)/bwidth)), histtype='step', label='lep 1', color='r')
            n2, bins2, _ = this_ax.hist(overflow_data2, bins=np.linspace(xmin,xmax,int((xmax-xmin)/bwidth)), histtype='step', label='lep 2', color='b')
            this_ax.legend()
        #     this_ax.xaxis.set_minor_formatter(plt.ScalarFormatter())
#             plt.minorticks_on()    
            plt.tight_layout()

        # End for loop over pT bins. 

        if (save_plots):
            pdf.savefig()  # saves the current figure into a pdf page

        plt.close()  # I think this is the end of the page.}